In [ ]:
from langchain.schema import Document
import json

# 파일 경로 
file_paths = {
    "term": "./metadata/term.json",
    "load_traffic_law": "./metadata/load_traffic_law.json",
    "modifier": "./metadata/modifier.json",
    "car_case": "./metadata/car_to_car.json",
    "precedent": "./metadata/precedent.json"
}

# 교통사고 케이스용 필드 상수
CASE_ID = "사건 ID"
CASE_TITLE = "사건 제목"
CASE_SITUATION = "사고상황"
BASE_RATIO = "기본 과실비율"
MODIFIERS = "케이스별 과실비율 조정예시"
LAW_REFERENCES = "관련 법규"
PRECEDENT = "참고 판례"
REASON = "기본 과실비율 해설"

# JSON 로드 함수
def load_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

# 리스트형 JSON 변환 (term, modifier, law_meta)
def convert_list_to_documents(data_list, doc_type):
    return [
        Document(page_content=json.dumps(item, ensure_ascii=False), metadata={"type": doc_type})
        for item in data_list
    ]

def convert_precedent_documents(data_list):
    return [
        Document(
            page_content=f"{item['court']} {item['case_id']} : {item['content']}",
            metadata={
                "court": item["court"],
                "case_id": item["case_id"],
            }
        ) for item in data_list
    ]

def convert_car_case_documents(data_list):
    documents = []

    def safe_value(value):
        if isinstance(value, list):
            return ", ".join(map(str, value))
        elif isinstance(value, dict):
            return json.dumps(value, ensure_ascii=False)
        elif value is None:
            return ""  # null도 허용 안 되므로 빈 문자열로 처리
        else:
            return str(value)

    for item in data_list:
        if not isinstance(item, dict):
            continue

        # page_content는 원본 전체 JSON 문자열
        content = json.dumps(item, ensure_ascii=False)

        # 기본 과실비율 해설이 리스트일 수 있음 → 문자열로 병합
        reason = item.get(REASON)
        if isinstance(reason, list):
            reason = "\n".join(map(str, reason))

        metadata = {
            "type": "car_case",
            "id": safe_value(item.get(CASE_ID)),
            "title": safe_value(item.get(CASE_TITLE)),
            "situation": safe_value(item.get(CASE_SITUATION)),
            "base_ratio": safe_value(item.get(BASE_RATIO)),
            "modifiers": safe_value(item.get(MODIFIERS)),
            "load_traffic_law": safe_value(item.get(LAW_REFERENCES)),
            "precedent": safe_value(item.get(PRECEDENT)),
            "reason": safe_value(reason)
        }

        documents.append(Document(page_content=content, metadata=metadata))
    return documents

# 도로교통법 law JSON → 문서화
def convert_law_json_to_documents(data_dict):
    documents = []

    def normalize(item):
        return json.dumps(item, ensure_ascii=False) if isinstance(item, dict) else str(item)

    for law_name, content in data_dict.items():
        if isinstance(content, dict):
            for clause, text in content.items():
                lines = [normalize(x) for x in (text if isinstance(text, list) else [text])]
                full_text = f"{law_name} {clause}\n" + "\n".join(lines)
                documents.append(Document(page_content=full_text, metadata={"type": "load_traffic_law"}))
        else:
            lines = [normalize(x) for x in (content if isinstance(content, list) else [content])]
            full_text = f"{law_name}\n" + "\n".join(lines)
            documents.append(Document(page_content=full_text, metadata={"type": "load_traffic_law"}))
    
    return documents
    

import random
json_precedent = load_json(file_paths["precedent"])
random_precedent = random.sample(json_precedent, 10)
# for precedent in random_precedent:
#     print(precedent['court'])
#     print(precedent['case_id'])

# 문서화 실행
term_docs = convert_list_to_documents(load_json(file_paths["term"]), "term")
modifier_docs = convert_list_to_documents(load_json(file_paths["modifier"]), "modifier")
precedent_docs = convert_precedent_documents(random_precedent)
car_case_docs = convert_car_case_documents(load_json(file_paths["car_case"]))
load_traffic_law_docs = convert_law_json_to_documents(load_json(file_paths["load_traffic_law"]))


# 전체 문서 리스트
#all_docs = term_docs + modifier_docs + car_case_docs + precedent_docs + load_traffic_law_docs
all_docs = precedent_docs

### 임베딩 모델

In [ ]:
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings()

In [ ]:
# from langchain_community.embeddings import HuggingFaceEmbeddings

# embedding_model = HuggingFaceEmbeddings(
#     model_name="jhgan/ko-sbert-nli",  # 허깅페이스 모델명
#     model_kwargs={"device": "cpu"},    # GPU 사용 시 "cuda"로 변경
#     encode_kwargs={"normalize_embeddings": True}  # 임베딩 정규화
# )

### chromadb

In [ ]:
from langchain.vectorstores import Chroma

chroma_vector_store = Chroma.from_documents(
    documents=all_docs,
    embedding=embedding_model,
    persist_directory="./chroma_db"
)

chroma_vector_store.persist()

chroma_vector_store.get()

### FAISS 적용

In [ ]:
from langchain.vectorstores import FAISS

faiss_vector_store = FAISS.from_documents(
    documents=all_docs,
    embedding=embedding_model
)

# 저장
faiss_vector_store.save_local(folder_path="./faiss_db")

In [ ]:
# 저장된 FAISS DB 불러오기
faiss_vector_store = FAISS.load_local(
    folder_path="./faiss_db",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 1. LLM 설정
model = ChatOpenAI(model='gpt-4o', temperature=0.5)

# 2. 프롬프트 템플릿
prompt = PromptTemplate(
    template="""
아래 문서 내용을 바탕으로 사용자가 물어본 용어나 법률 조항, 판례에 대해 정확하고 간결하게 설명해 주세요.

질문: {question}

문서: {context}

답변 형식:
- 용어/조항 정의: [정확한 설명]
- 출처가 명시된 경우: 관련 법률/조문 번호/판례명을 반드시 포함

답변:
""",
    input_variables=["question", "context"]
)

### Chroma

In [ ]:
from langchain.chains import RetrievalQA

# 3. 리트리버 설정
retriever_chroma = chroma_vector_store.as_retriever(search_kwargs={"k": 3})

# 4. QA 체인 구성
qa_chain_chroma = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever_chroma,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)

### FAISS

In [ ]:
from langchain.chains import RetrievalQA

# 3. 리트리버 설정
retriever_faiss = faiss_vector_store.as_retriever(search_kwargs={"k": 3})

# 4. QA 체인 구성
qa_chain_faiss = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever_faiss,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:
for precedent in random_precedent:
    # 5. 실제 질의 실행
    query = f"{precedent['court']} {precedent['case_id']} 판례 내용 알려줘"

    res_chroma = qa_chain_chroma.invoke({"query": query})
    res_faiss = qa_chain_faiss.invoke({"query": query})

    # 6. 출력
    print("✅ Chroma 답변:\n", res_chroma["result"])
    print("="*50)
    print("✅ FAISS 답변:\n", res_faiss["result"])
    print("\n\n")

In [ ]:
query = "전주지방법원 2009가단22343 판례 내용 알려줘"

res_chroma = qa_chain_chroma.invoke({"query": query})

# 6. 출력
print("✅ 필터 적용 전 >> Chroma 답변:\n", res_chroma["result"])



from langchain.chains import RetrievalQA

# 3. 리트리버 설정
retriever_chroma = chroma_vector_store.as_retriever(
    search_kwargs={
        "k": 5,
        "filter": {
            "case_id": "2009가단22343"
        }
    }
)

# 4. QA 체인 구성
qa_chain_chroma = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever_chroma,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)

res_chroma = qa_chain_chroma.invoke({"query": query})

# 6. 출력
print("✅ 필터 적용 후 >> Chroma 답변:\n", res_chroma["result"])